In [1]:
#AS faiss wasnt working for me i decided to use chromadb

In [1]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
from langchain import PromptTemplate

# Original prompt template
prompt_template = """
    This is a chatbot designed to answer questions about Aymen. 
    It Uses the provided documents to answer questions accurately and politely."
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(template=prompt_template)

In [3]:
# Define the context and question
context = """
Aymen is a Master's student in DSAI at AIT. 
They graduated with a bachelors degree in Electrical Engineering.
"""
question = "What is Aymen's highest level of education?"

# Format the prompt
formatted_prompt = PROMPT.format(context=context, question=question)
print(formatted_prompt)

This is a chatbot designed to answer questions about Aymen. 
    It Uses the provided documents to answer questions accurately and politely."
    
Aymen is a Master's student in DSAI at AIT. 
They graduated with a bachelors degree in Electrical Engineering.

    Question: What is Aymen's highest level of education?
    Answer:


In [4]:
#Using the txt documents that include my information

In [5]:
from langchain.document_loaders import TextLoader

# Load education document
education_loader = TextLoader('education.txt')
education = education_loader.load()

# Load beliefs document
beliefs_loader = TextLoader('beliefs.txt')
beliefs = beliefs_loader.load()

# Load academics document
academics_loader = TextLoader('academics.txt')
academics = academics_loader.load()

In [6]:
# Combine all documents into a single list
documents = education + beliefs + academics

# Check the number of documents loaded
print(f"Number of documents loaded: {len(documents)}")

# Inspect the first document (optional)
print(documents[0].page_content)

Number of documents loaded: 3
Education and Background

Personal Information:
- Name: Aymen Zubair Qureshi
- Date of Birth: August 15, 2002
- Age: 22

Education:
1. Master of Science in Data Science and Artificial Intelligence:
   - Institution: Asian Institute of Technology, Thailand
   - Duration: August 2024 – Present
   - Focus Areas: Machine Learning, Deep Learning, AI Ethics

2. Bachelor of Science in Electrical Engineering (Communication Major):
   - Institution: University of Engineering and Technology (UET), Pakistan
   - Duration: 2020–2024
   - Focus Areas: Signal Processing, Communication Systems

Highest Level of Education: Master’s Degree
Field of Study: Data Science and Artificial Intelligence


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,  # Size of each chunk (in characters)
    chunk_overlap=100  # Overlap between chunks (in characters)
)

# Split the documents
docs = text_splitter.split_documents(documents)

# Check the number of chunks
print(f"Number of chunks: {len(docs)}")

# Inspect the first chunk (optional)
print(docs[0].page_content)

Number of chunks: 6
Education and Background

Personal Information:
- Name: Aymen Zubair Qureshi
- Date of Birth: August 15, 2002
- Age: 22

Education:
1. Master of Science in Data Science and Artificial Intelligence:
   - Institution: Asian Institute of Technology, Thailand
   - Duration: August 2024 – Present
   - Focus Areas: Machine Learning, Deep Learning, AI Ethics

2. Bachelor of Science in Electrical Engineering (Communication Major):
   - Institution: University of Engineering and Technology (UET), Pakistan
   - Duration: 2020–2024
   - Focus Areas: Signal Processing, Communication Systems

Highest Level of Education: Master’s Degree
Field of Study: Data Science and Artificial Intelligence


In [8]:
docs[1]

Document(metadata={'source': 'beliefs.txt'}, page_content='Personal Beliefs and Cultural Perspectives\n\nCore Beliefs about Technology:\nI believe that technology has the power to transform societies by improving access to education, healthcare, and economic opportunities. However, it is crucial to ensure that technological advancements are inclusive and ethical, addressing the needs of all communities, especially marginalized ones.')

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize the embedding model
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": device}
)

print("Embedding model initialized successfully!")

/tmp/ipykernel_81755/438159016.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Embedding model initialized successfully!


In [10]:
import os
from langchain.vectorstores import Chroma

# Define the path to save the vector store
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('Created vector store directory.')

# Create the Chroma vector store from documents
vectordb = Chroma.from_documents(
    documents=docs,  # Your text chunks
    embedding=embedding_model,  # Your embedding model
    persist_directory=os.path.join(vector_path, 'chroma_db')  # Save to disk
)

print("Vector store created and saved successfully!")

Vector store created and saved successfully!


In [11]:
from langchain.vectorstores import Chroma

# Define the path to the saved vector store
vector_path = '../vector-store'

# Load the Chroma vector store
vectordb = Chroma(
    persist_directory=os.path.join(vector_path, 'chroma_db'),
    embedding_function=embedding_model  # Your embedding model
)

print("Vector store loaded successfully!")

Vector store loaded successfully!


/tmp/ipykernel_81755/616098682.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [12]:
# Create a retriever
retriever = vectordb.as_retriever()

# Test the retriever
query = "What is Aymens education level?"
relevant_docs = retriever.get_relevant_documents(query)

# Print the results
print(f"Relevant documents for query: '{query}'")
for doc in relevant_docs:
    print(doc.page_content)

Relevant documents for query: 'What is Aymens education level?'
Education and Background

Personal Information:
- Name: Aymen Zubair Qureshi
- Date of Birth: August 15, 2002
- Age: 22

Education:
1. Master of Science in Data Science and Artificial Intelligence:
   - Institution: Asian Institute of Technology, Thailand
   - Duration: August 2024 – Present
   - Focus Areas: Machine Learning, Deep Learning, AI Ethics

2. Bachelor of Science in Electrical Engineering (Communication Major):
   - Institution: University of Engineering and Technology (UET), Pakistan
   - Duration: 2020–2024
   - Focus Areas: Signal Processing, Communication Systems

Highest Level of Education: Master’s Degree
Field of Study: Data Science and Artificial Intelligence
Education and Background

Personal Information:
- Name: Aymen Zubair Qureshi
- Date of Birth: August 15, 2002
- Age: 22

Education:
1. Master of Science in Data Science and Artificial Intelligence:
   - Institution: Asian Institute of Technology, T

/tmp/ipykernel_81755/1207648260.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)


In [13]:
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI

# Step 1: Initialize memory
history = ChatMessageHistory()

# Add user and AI messages
history.add_user_message("hi")
history.add_ai_message("What's up?")
history.add_user_message("How are you?")
history.add_ai_message("I'm quite good. How about you?")

# Print the conversation history
print("Chat History:")
for message in history.messages:
    print(f"{message.type}: {message.content}")

# Step 2: Use ConversationBufferMemory
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"output": "What's up?"})
memory.save_context({"input": "How are you?"}, {"output": "I'm quite good. How about you?"})

# Print the memory buffer
print("\nMemory Buffer:")
print(memory.load_memory_variables({}))

Chat History:
human: hi
ai: What's up?
human: How are you?
ai: I'm quite good. How about you?

Memory Buffer:
{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}


/tmp/ipykernel_81755/3977671842.py:20: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain import HuggingFacePipeline
import re

# Step 1: Load the tokenizer and model
model_id = "google/flan-t5-large"  # Use Flan-T5 instead of fastchat-t5
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Step 2: Test the tokenizer and model
text = "What is Aymen's age?"
tokens = tokenizer(text, return_tensors="pt")
print(f"Tokens: {tokens}")

output = model.generate(tokens["input_ids"], max_new_tokens=50)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Model Output: {decoded_output}")

# Step 3: Create a text generation pipeline
pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    model_kwargs={
        "temperature": 0,
        "repetition_penalty": 1.5
    }
)

# Step 4: Wrap the pipeline in a LangChain HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

# Step 5: Test the LangChain pipeline
try:
    response = llm("What is Aymen's age?")
    print(f"LangChain Output: {response}")
except Exception as e:
    print(f"Error in LangChain pipeline: {e}")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

Tokens: {'input_ids': tensor([[ 363,   19,   71,   63,  904,   31,    7, 1246,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Device set to use cuda:0


Model Output: 66


/tmp/ipykernel_81755/1208319972.py:32: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipykernel_81755/1208319972.py:36: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm("What is Aymen's age?")


LangChain Output: 66


In [17]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

# Define the question generator
question_generator = LLMChain(
    llm=llm,  # Your language model
    prompt=CONDENSE_QUESTION_PROMPT,  # Prompt to rephrase follow-up questions
    verbose=True  # For debugging
)

In [18]:
from langchain.chains.question_answering import load_qa_chain

# Define the document chain
doc_chain = load_qa_chain(
    llm=llm,  
    chain_type="stuff",  
    verbose=True  
)

/tmp/ipykernel_81755/517678038.py:4: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  doc_chain = load_qa_chain(


In [19]:
print(f"Context: {context}")
print(f"Question: {question}")

Context: 
Aymen is a Master's student in DSAI at AIT. 
They graduated with a bachelors degree in Electrical Engineering.

Question: What is Aymen's highest level of education?


In [20]:
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x79ffe31e4890>), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context')

In [21]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h: h
)
chain

/tmp/ipykernel_81755/75795424.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(
/tmp/ipykernel_81755/75795424.py:10: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~create_history_aware_retriever together with create_retrieval_chain (see example in docstring)` instead.
  chain = ConversationalRetrievalChain(


ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x79ffe31e4890>), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), question_generator=LLMCh

In [22]:
prompt_question = " What is aymens age?"
answer = chain({"question":prompt_question})
answer

/tmp/ipykernel_81755/4270988876.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chain({"question":prompt_question})
Token indices sequence length is longer than the specified maximum sequence length for this model (662 > 512). Running this sequence through the model will result in indexing errors




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Education and Background

Personal Information:
- Name: Aymen Zubair Qureshi
- Date of Birth: August 15, 2002
- Age: 22

Education:
1. Master of Science in Data Science and Artificial Intelligence:
   - Institution: Asian Institute of Technology, Thailand
   - Duration: August 2024 – Present
   - Focus Areas: Machine Learning, Deep Learning, AI Ethics

2. Bachelor of Science in Electrical Engineering (Communication Major):
   - Institution: University of Engineering and Technology (UET), Pakistan
   - Duration: 2020–2024
   - Focus Areas: Signal Processing, Communication Systems

Highest Level of Education: Master’s Degree
Field of Study: Data Science and Artificial 

{'question': ' What is aymens age?',
 'chat_history': [],
 'answer': '22',
 'source_documents': [Document(metadata={'source': 'education.txt'}, page_content='Education and Background\n\nPersonal Information:\n- Name: Aymen Zubair Qureshi\n- Date of Birth: August 15, 2002\n- Age: 22\n\nEducation:\n1. Master of Science in Data Science and Artificial Intelligence:\n   - Institution: Asian Institute of Technology, Thailand\n   - Duration: August 2024 – Present\n   - Focus Areas: Machine Learning, Deep Learning, AI Ethics\n\n2. Bachelor of Science in Electrical Engineering (Communication Major):\n   - Institution: University of Engineering and Technology (UET), Pakistan\n   - Duration: 2020–2024\n   - Focus Areas: Signal Processing, Communication Systems\n\nHighest Level of Education: Master’s Degree\nField of Study: Data Science and Artificial Intelligence'),
  Document(metadata={'source': 'education.txt'}, page_content='Education and Background\n\nPersonal Information:\n- Name: Aymen Zuba

In [23]:
import json
from langchain.chains import ConversationalRetrievalChain

# Step 1: Define the questions
questions = [
    "How old are you?",
    "What is your highest level of education?",
    "What major or field of study did you pursue during your education?",
    "How many years of work experience do you have?",
    "What type of work or industry have you been involved in?",
    "Can you describe your current role or job responsibilities?",
    "What are your core beliefs regarding the role of technology in shaping society?",
    "How do you think cultural values should influence technological advancements?",
    "As a master’s student, what is the most challenging aspect of your studies so far?",
    "What specific research interests or academic goals do you hope to achieve during your time as a master’s student?"
]

# Step 2: Initialize an empty list for question-answer pairs
qa_pairs = []

# Step 3: Generate answers using the chatbot
for question in questions:
    response = chain({"question": question})
    answer = response["answer"]
    qa_pairs.append({
        "question": question,
        "answer": answer
    })

# Step 4: Convert to JSON and save to a file
json_output = json.dumps(qa_pairs, indent=2)
with open("question_answer_pairs.json", "w") as f:
    f.write(json_output)

print("Question-answer pairs saved to question_answer_pairs.json")



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content=' What is aymens age?', additional_kwargs={}, response_metadata={}), AIMessage(content='22', additional_kwargs={}, response_metadata={})]
Follow Up Input: How old are you?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Education and Background

Personal Information:
- Name: Aymen Zubair Qureshi
- Date of Birth: August 15, 2002
- Age: 22

Education:
1. Master of Science in Data Science and Artificial Intelligence:
   - Institution: Asia

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



> Finished chain.

> Finished chain.

> Finished chain.


> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What is your highest level of education?', additional_kwargs={}, response_metadata={}), AIMessage(content='Master’s Degree', additional_kwargs={}, response_metadata={}), HumanMessage(content='What major or field of study did you pursue during your education?', additional_kwargs={}, response_metadata={}), AIMessage(content='data science and AI', additional_kwargs={}, response_metadata={}), HumanMessage(content='How many years of work experience do you have?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am currently a student and do not have any formal work experience', additional_kwargs={}, response_metadata={})]
Fo